# Elliptic Curve Menezes-Qu-Vanstone (ECMQV)

ECMQV is an extension of the regular Diffie-Hellman key agreement protocol.

In [1]:
import random
import math
from typing import Tuple
import hmac
import hashlib
from Crypto.Cipher import AES

# Addition Law

In [2]:
# y^2 = x^3 + a*x + b

In [3]:
def add_points(P, Q, p):
    x1, y1 = P
    x2, y2 = Q
    
    if x1 == x2 and y1 == y2:
        beta = (3*x1*x2 + a) * pow(2*y1, -1, p)
    else:
        beta = (y2 - y1) * pow(x2 - x1, -1, p)
    
    x3 = (beta*beta - x1 - x2) % p
    y3 = (beta * (x1 - x3) - y1) % p
    
    is_on_curve((x3, y3), p)
        
    return x3, y3

def is_on_curve(P, p):
    x, y = P
    assert (y*y) % p == ( pow(x, 3, p) + a*x + b ) % p
    
def apply_double_and_add_method(G, k, p):
    target_point = G
    
    k_binary = bin(k)[2:] #0b1111111001
    
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
        
        # doubling - always
        target_point = add_points(target_point, target_point, p)
        
        if current_bit == "1":
            target_point = add_points(target_point, G, p)
    
    is_on_curve(target_point, p)
    
    return target_point

# Curve Configuration

In [4]:
# Secp256k1

# curve formula
a = 0; b = 7

# base point
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424)

# modulo
p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)

# order
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

# cofactor
h = 1

In [5]:
is_on_curve(G, p)

In [6]:
is_on_curve(P = apply_double_and_add_method(G = G, k = n - 1, p = p), p = p)

# ECMQV

In [7]:
def bar(P: Tuple) -> int:
    l = math.ceil( (math.floor(math.log(n, 2)) + 1) / 2 )
    x, y = P
    P_bar = ( x % pow(2, l) ) + pow(2, l)
    return P_bar

def derive_keys(T):
    tx, ty = T
    
    tx_binary = bin(tx)[2:]
    
    #192-bits
    tx_binary_cropped = tx_binary[0:192]
    
    tx_restored = int(tx_binary_cropped, 2)
    
    #sha-256
    hash_hex = hashlib.sha256(str.encode(str(tx_restored))).hexdigest()
    hash_binary = bin(int(hash_hex, 16))[2:]
    
    k1 = int(hash_binary[0:128], 2).to_bytes(16, byteorder="big")
    k2 = int(hash_binary[128:], 2).to_bytes(16, byteorder="big")
    
    return k1, k2

def find_mac(message, key):
    return hmac.new(key, message, hashlib.sha256).hexdigest()

# Private - Public Key Pairs

In [8]:
# private key of Alice
ka = random.getrandbits(256)

# public key of Alice
Qa = apply_double_and_add_method(G = G, k = ka, p = p)

# random key of Alice (secret)
ra = random.getrandbits(256)

# random point of Alice (public)
Ra = apply_double_and_add_method(G = G, k = ra, p = p)

In [9]:
# private key of Bob
kb = random.getrandbits(256)

# public key of Bob
Qb = apply_double_and_add_method(G = G, k = kb, p = p)

# random key of Bob (secret)
rb = random.getrandbits(256)

# random point of Bob (public)
Rb = apply_double_and_add_method(G = G, k = rb, p = p)

# Signatures

In [10]:
# Alice sends sa to Bob
sa = ( ra + bar(Ra) * ka ) % n

In [11]:
# Bob sends sb to Alice
sb = ( rb + bar(Rb) * kb ) % n

# Key Exchange

In [12]:
# Alice calculate Ja
# Ja = h x sa x (Rb + Rb_bar x Qb)
Ja = apply_double_and_add_method(G = Qb, k = bar(Rb), p = p)
Ja = add_points(P = Rb, Q = Ja, p = p)
Ja = apply_double_and_add_method(G = Ja, k = h*sa, p = p)

In [13]:
Ja

(82965357047056979595736171426934942749871555929481247850580347056485951663139,
 14961028411908453185605960697534623645110006497253043979079201288235037500611)

In [14]:
# Bob calculates Jb
# Jb = h x sb x (Ra + Ra_bar x Qa)
Jb = apply_double_and_add_method(G = Qa, k = bar(Ra), p = p)
Jb = add_points(P = Ra, Q = Jb, p = p)
Jb = apply_double_and_add_method(G = Jb, k = h*sb, p = p)

In [15]:
Jb

(82965357047056979595736171426934942749871555929481247850580347056485951663139,
 14961028411908453185605960697534623645110006497253043979079201288235037500611)

In [16]:
assert Ja == Jb

# Additional Security Layer with MAC

Once Alice and Bob shared same J keys (Ja & Jb), they can feed this

to a key derivation function and derive 2 keys: k1 and k2.

They will use k1 to generate message authentication code (MAC)

If MACs can be verified, then they will use k2 as AES-128 key.

In [17]:
# Bob uses KDF and gets k1, k2 pair
k1b, k2b = derive_keys(Jb)

# Alice uses KDF to find k1, k2 pair
k1a, k2a = derive_keys(Ja)

In [18]:
# Bob finds MAC for the message with k2 key
# Notice that an attacker does not know k2, so the attacker cannot find tb
msg = f"2BobAlice{Rb[0]}{Rb[1]}{Ra[0]}{Ra[1]}"
tb = find_mac(message = bytes(msg, "utf-8"), key = k2b)

In [19]:
# Alice uses k2 to validate tb coming from Bob
msg = f"2BobAlice{Rb[0]}{Rb[1]}{Ra[0]}{Ra[1]}"
t = find_mac(message = bytes(msg, "utf-8"), key = k2a)
assert t == tb

# Then she finds the mac of the message with k2 key
# Notice that Bob already knows k2, so he can validate ta
msg = f"2AliceBob{Ra[0]}{Ra[1]}{Rb[0]}{Rb[1]}"
ta = find_mac(message = bytes(msg, "utf-8"), key = k2a)
# Alice sends ta to Bob

In [20]:
# Bob verifies ta coming from Alice
msg = f"2AliceBob{Ra[0]}{Ra[1]}{Rb[0]}{Rb[1]}"
t = find_mac(message = bytes(msg, "utf-8"), key = k2b)
assert t == ta

In [21]:
# Once Alice and Bob validate message authentication codes
# they can use k1 as AES-128 key
print(f"Session key is {k1a} ({len(k2a)*8} bits)")

Session key is b'\xe1\x89OP\xccBx\xef\xc0<\xfe\r\xfc\x05\xbeD' (128 bits)


# Symmetric Key Encryption

In [22]:
# bob will encrypt a message with k1
msg = "attack tomorrow!"
obj_bob = AES.new(k1b)
c = obj_bob.encrypt(msg)
print(f"ciphertext is {c}")

ciphertext is b'\x1c.y\xf5\xefK\xc6T(\x07\xc1\xaf6W\x9fh'


In [23]:
# alice will decrypt a message with k1
obj_alice = AES.new(k1a)
plaintext = obj_alice.decrypt(c)
print(f"restored plaintext is {plaintext}")

restored plaintext is b'attack tomorrow!'


# Proof

Alice calculates:

    Ja = h x sa x (Rb + Rb_bar x Qb)

    Ja = h x sa x (rb x G + Rb_bar x kb x G)

    Ja = h x sa x (rb + Rb_bar x kb) x G

    Ja = h x sa x sb x G

Bob calculates: 

    Jb = h x sb x (Ra + Ra_bar x Qa)

    Jb = h x sb x (ra x G + Ra_bar x ka x G)

    Jb = h x sb x (ra + Ra_bar x ka) x G

    Jb = h x sb x sa x G

So, Alice and Bob must calculate same point!

    Ja = Jb